In [1]:
import pandas as pd
import numpy as np
from scipy.io import wavfile
from tqdm import tqdm

In [2]:
df = pd.read_csv('Passaros/cantos2.csv', encoding='latin-1', sep=';')
df.head()

,WAV_total,Especie,Canto,mp3_total,se_pendrive,sub,se_15+,esp_sub
0,59585ea4b32685f8e0328334b8fb966a.wav,Crypturellus,song,59585ea4b32685f8e0328334b8fb966a.mp3,sim,albigularis,sim,Crypturellus_albigularis
1,62843f1aae85dd853ac15d0db36c095b.wav,Crypturellus,song,62843f1aae85dd853ac15d0db36c095b.mp3,sim,albigularis,sim,Crypturellus_albigularis
2,323eed79164501a3b70838c319638159.wav,Crypturellus,song,323eed79164501a3b70838c319638159.mp3,sim,albigularis,sim,Crypturellus_albigularis
3,ca14c8ee56a94062f8436ea02a970971.wav,Crypturellus,song,ca14c8ee56a94062f8436ea02a970971.mp3,sim,albigularis,sim,Crypturellus_albigularis
4,cc8019aad97ab259345ea74e510a238b.wav,Crypturellus,song,cc8019aad97ab259345ea74e510a238b.mp3,sim,albigularis,sim,Crypturellus_albigularis


In [3]:
df.rename(columns={'esp_sub':'label'}, inplace=True)
df.rename(columns={'WAV_total':'fname'}, inplace=True)

In [4]:
print(len(df))
df = df[df['se_15+']=='sim']
print(len(df))

482
482


In [5]:
print(len(df))
df = df[df['se_pendrive']=='sim']
print(len(df))

482
482


In [6]:
# esp = pd.DataFrame(df.label.value_counts())
# esp = list(esp.head(30).index)
# len(esp)
# df = df[df.label.isin(esp)]

In [7]:
# for a in ['Myrmophylax_melanura','Syndactyla_rufosuperciliata', 'Xiphorhynchus_fuscus', 'Cichlocolaptes_holti']:
#     print(a, len(df[df.label==a]))

In [8]:
# for c in ['Sclerurus_cearensis', 'Syndactyla_rufosuperciliata', 'Microrhopias_bicolor']:
#     df = df[df.label != c]

In [9]:
len(df['label'].value_counts())

23

In [10]:
df = df[['fname', 'label']]
df

,fname,label
0,59585ea4b32685f8e0328334b8fb966a.wav,Crypturellus_albigularis
1,62843f1aae85dd853ac15d0db36c095b.wav,Crypturellus_albigularis
2,323eed79164501a3b70838c319638159.wav,Crypturellus_albigularis
3,ca14c8ee56a94062f8436ea02a970971.wav,Crypturellus_albigularis
4,cc8019aad97ab259345ea74e510a238b.wav,Crypturellus_albigularis
...,...,...
477,2f50721abc871ee082f397f3029916f7.wav,Myrmophylax_melanura
478,8e0fd92f90b19046adf93f0a2ed42ca7.wav,Myrmophylax_melanura
479,5fae2f1c2f098beab8d62dbae494af7c.wav,Myrmophylax_melanura
480,a6a72518c5c2f7bf412670f243f4fd17.wav,Myrmophylax_melanura


In [11]:
df.fname = df.fname.apply(lambda x: x[:-4]+'_filtrado.wav')

In [12]:
df.set_index('fname', inplace=True)

In [13]:
# Import the AudioSegment class for processing audio and the 
# split_on_silence function for separating out silent chunks.
from pydub import AudioSegment
from pydub.silence import split_on_silence
from tqdm import tqdm
import os

# Define a function to normalize a chunk to a target amplitude.
def match_target_amplitude(aChunk, target_dBFS):
    ''' Normalize given audio chunk '''
    change_in_dBFS = target_dBFS - aChunk.dBFS
    return aChunk.apply_gain(change_in_dBFS)

# Create a silence chunk that's 0.5 seconds (or 500 ms) long for padding.
# silence_chunk = AudioSegment.silent(duration=150)

#audio_dir = caminho pra pasta onde estão os arquivos
audio_dir = 'Passaros/'
#audio_dir2 = caminho pra pasta onde quer salvar os arquivos
audio_dir2 = 'sem_silencio/'

for fname in tqdm(df.index):
    if fname not in os.listdir(audio_dir2):
        if fname[-4:] == '.wav':
            audio_chunk = 0
            # Load your audio.
            song = AudioSegment.from_wav(audio_dir+fname)

            # Split track where the silence is 2 seconds or more and get chunks using 
            # the imported function.
            chunks = split_on_silence (
                # Use the loaded audio.
                song, 
                # Specify that a silent chunk must be at least 2 seconds or 2000 ms long.
                min_silence_len = 1000,
                # Consider a chunk silent if it's quieter than -16 dBFS.
                # (You may want to adjust this parameter.)
                silence_thresh = -40
            )

            # Process each chunk with your parameters
            if len(chunks) > 1:
                for chunk in chunks:
                    if audio_chunk == 0:
                        audio_chunk =  chunk
                    else:
                        audio_chunk = audio_chunk + chunk
            else:
                audio_chunk = chunk
            # Normalize the entire chunk.
            normalized_chunk = match_target_amplitude(audio_chunk, -20.0)
            normalized_chunk.export(audio_dir2+fname, bitrate='192k', format="wav")

100%|██████████| 482/482 [00:00<00:00, 2590.04it/s]


In [14]:
diretorio = 'sem_silencio/'
for f in tqdm(df.index):
    rate, signal = wavfile.read(diretorio+f)
    df.at[f, 'length'] = signal.shape[0]/rate

100%|██████████| 482/482 [00:01<00:00, 406.74it/s]


In [15]:
a = pd.DataFrame(df['length'].groupby(df['label']).sum())
soma = a.length.sum()
a['porc'] = round(a.length/soma*100, 3)
a = a.sort_values(by=['length'])
a

,length,porc
label,,
Brotogeris_chiriri,66.251882,1.115
Myrmophylax_melanura,78.367120,1.319
Patagioenas_plumbea,125.045941,2.105
Microrhopias_bicolor,125.202086,2.107
Xiphorhynchus_tenuirostris,125.227029,2.108
Pygiptila_stellaris,127.588707,2.147
Sclerurus_cearensis,151.660726,2.552
Xiphorhynchus_atlanticus,157.403991,2.649
Crypturellus_albigularis,159.162948,2.679


In [16]:
print(df.length.describe())
del[df['length']]

count    482.000000
mean      12.327410
std       21.226810
min        0.113016
25%        1.448027
50%        5.369524
75%       14.391202
max      216.411973
Name: length, dtype: float64


In [17]:
# print(len(df[df.length>94])/len(df))
# print(len(df[df.length<5])/len(df))
# print(df.length.describe())
# df = df[df.length>5]
# df = df[df.length<94]
# print(df.length.describe())

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.index, df.label, test_size=0.2, random_state=100, stratify=df.label)

In [19]:
treino = pd.DataFrame(y_train, X_train)
treino

,label
fname,
80c60a2a625beeb09c36f7fd6c85e87d_filtrado.wav,Megascops_usta
85f8eb6dd1db20c859ab9e63d73d401a_filtrado.wav,Xiphorhynchus_eytoni
4df06758a9fc79eedb45d5c8fcf299f4_filtrado.wav,Cichlocolaptes_holti
1b58ba9cfbf8da4fab57f84d404f8735_filtrado.wav,Sittasomus_sylviellus
6f90de95a2b4439584c711bf2e38502b_filtrado.wav,Cichlocolaptes_holti
...,...
9dc5a3f981af1d785d6c7088a2f655fa_filtrado.wav,Sclerurus_macconnelli
41a4b7ddbed84594129c96b9a7373017_filtrado.wav,Xiphorhynchus_guttatoides
2c9107f3e7640a858af638b0336b8c5f_filtrado.wav,Microrhopias_bicolor


In [20]:
teste = pd.DataFrame(y_test, X_test)
teste

,label
fname,
9f0acb1b2547343b53115635eb119aca_filtrado.wav,Xiphorhynchus_fuscus
13e85a88d9f2d6dbcc1f57df7fefffc8_filtrado.wav,Trogon_aurantius
3b824e02b7c19c84630477daef505d66_filtrado.wav,Crypturellus_obsoletus
7b5c641d9a89adb5e306cbb1de77df76_filtrado.wav,Sclerurus_cearensis
6b4479b29dfe82eda127bab4a3ad6087_filtrado.wav,Sittasomus_sylviellus
...,...
323eed79164501a3b70838c319638159_filtrado.wav,Crypturellus_albigularis
afcd7894354482c2aecb4479e190cc0e_filtrado.wav,Phaethornis_eurynome
81d79e7b1191c8f2f7bddc4fc6a30310_filtrado.wav,Myrmophylax_melanura


In [21]:
len(teste.label.value_counts())
len(treino.label.value_counts())

23

In [22]:
teste.to_csv('teste_f.csv')
treino.to_csv('treino_f.csv')

In [23]:
len(teste)/len(treino)

0.2519480519480519

In [24]:
len(teste)

97